bnds = (,(0,x))

We will here work with a benchmark model of insurance policies, Mossin's model.

We are dealing with an agent wanting to get an insurance. 

The agent holds assets **y** and may suffer a monetary loss **x**.
For now we assume that the loss is incurred with probability **p**.

Utility of assets is given by $u(\cdot)$.
If the agent is not insured, expected value is 
$$
\begin{aligned}
V_{0}=pu(y-x)+(1-p)(y)
\end{aligned}
$$

The agent may however obtain an insurance contract to mitigate a loss.

The contract consists of a coverage amount **q** and a premium $\pi$.

The coverage cannot exceed the loss so $q\in[0,x]$ and $\pi$
must be paid irrespective of whether a loss was incurred.

To not go broke, the insurance company must require higher premium for
better coverage.
It therefore implements a premium policy 
$$
\begin{aligned}
\pi(p,q)=pq
\end{aligned}
$$
An insured agent will thus have expected utility 
$$
\begin{aligned}
V(q;\pi)=pu(y-x+q-\pi(p,q))+(1-p)u(y-\pi(p,q))$
\end{aligned}
$$
To solve the model numerically, we let $u$ be given by the function 
$$
\begin{aligned}
u(z)=\frac{z^{1+\vartheta}}{1+\vartheta}
\end{aligned}
$$
And use the following parameterization:

$$
\begin{aligned}
y=1,\thinspace\thinspace p=0.2,\thinspace\thinspace\vartheta=-2
\end{aligned}
$$

In [89]:
# magic to reload modules automatically
%load_ext autoreload
%autoreload 2

# standard imports
import numpy as np
from scipy import optimize
from scipy.optimize import minimize
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [90]:
def u(z,vartheta):
        return (z)**(1+vartheta)/(1+vartheta)

In [102]:
def v_0(p,u,y,x):
    return p*u(y-x) + (1-p)*y

def v(p,u,y,x,q):
    pi =p*q
    return p*u(y-x+q-pi) + (1-p)*u(y-pi)


**Question 1**

 Construct a function that takes $(x,y,p)$ as arguments and returns the agents' optimal insurance coverage: 
\begin{align}
q^{\star}=\textrm{argmax}_{q\in[0,x]}V(q;\pi)
\end{align}

Calculate and plot $q^{\star}$ for **x** in the range $[0.01,0.9]$.

What does the graph suggest about the general solution to the agent's problem?

In [93]:
y = 1
p = 0.2
vartheta = -2

In [98]:
#def objective(v):
    #x = v[0]
    #q = v[1]
    #pi = p*q
    #return -(p*u(y-x+q-pi) + (1-p)*u(y-pi))  

#b1 = (0,None)
#b2 = (0,x)
#bounds = (b1,b2)
#constrain = {'type': 'ineq', 'fun': const1(v)}
#x0 = [0.2,0.2]
#sol = minimize(objective,x0,method='SLSQ',bounds=bnds)

In [112]:
objective = lambda v: -(p*u(y-v[0]+v[1]-pi) + (1-p)*u(y-pi))
x = v[0]
q = v[1]
pi = p*q
cons = {'type': 'ineq', 'fun': lambda x: v[0]-v[1]}
b1 = (0,None)
b2 = (0,v[0])
bnds = (b1,b2)
res = minimize(objective,(0.2,0.2),bounds=bnds,constrains=cons)
    

TypeError: 'function' object is not subscriptable